In [8]:
import pandas as pd
from datetime import datetime

# 1. Lade die Vorhersage-Datei
predictions = pd.read_csv('/workspaces/bakery_sales_prediction/3_Model/prediction_neural_net_v2.csv')
print("Vorhersage-Datei geladen:")
print(predictions.head())
print(f"Shape: {predictions.shape}")

# 2. Lade sample_submission für die korrekte Reihenfolge
sample_submission = pd.read_csv('/workspaces/bakery_sales_prediction/5_Datasets/sample_submission.csv')
print("\nSample submission geladen:")
print(sample_submission.head())
print(f"Shape: {sample_submission.shape}")

# 3. Überprüfe ID-Formate
print("\nID-Typ in sample_submission:", sample_submission['id'].dtype)

# 4. Warengruppen-Mapping
warengruppen_mapping = {
    'Brot': 1,
    'Brötchen': 2, 
    'Croissant': 3,
    'Konditorei': 4,
    'Kuchen': 5,
    'Saisonbrot': 6
}

# Invertiertes Mapping für die Rückkonvertierung
warengruppen_reverse_mapping = {v: k for k, v in warengruppen_mapping.items()}

# 5. Konvertiere Datum zu datetime falls nötig
predictions['Datum'] = pd.to_datetime(predictions['Datum'])

# 6. Erzeuge ID im Format YYMMDDW
def generate_id(row):
    date = row['Datum']
    warengruppe_code = warengruppen_mapping[row['Warengruppe_Name']]
    
    # Format: YYMMDDW
    year = str(date.year)[-2:]  # letzten 2 Stellen
    month = f"{date.month:02d}"
    day = f"{date.day:02d}"
    
    return int(f"{year}{month}{day}{warengruppe_code}")  # Als Integer zurückgeben

predictions['id'] = predictions.apply(generate_id, axis=1)

# 7. Bereite finale Submission vor
submission = predictions[['id', 'Umsatz']].copy()

# 8. Identifiziere fehlende IDs und analysiere diese
missing_ids_before = set(sample_submission['id']) - set(submission['id'])
print(f"\nAnzahl fehlender IDs vor Korrektur: {len(missing_ids_before)}")

# 9. NEUE STRATEGIE: Analysiere die fehlenden IDs und fülle sie mit ähnlichen Werten
if missing_ids_before:
    # Funktion zum Extrahieren von Datum und Warengruppe aus ID
    def extract_info_from_id(id_val):
        id_str = str(id_val).zfill(7)  # Stelle sicher, dass die ID 7 Stellen hat
        year = int("20" + id_str[0:2])
        month = int(id_str[2:4])
        day = int(id_str[4:6])
        warengruppe = int(id_str[6])
        return pd.Timestamp(year, month, day), warengruppe

    # Erstelle einen Rahmen mit allen fehlenden IDs
    missing_data = []
    
    for missing_id in missing_ids_before:
        date, warengruppe_code = extract_info_from_id(missing_id)
        warengruppe_name = warengruppen_reverse_mapping.get(warengruppe_code, "Unbekannt")
        
        # Versuche, ähnliche Daten zu finden (gleicher Wochentag und gleiche Warengruppe)
        similar_data = predictions[
            (predictions['Datum'].dt.dayofweek == date.dayofweek) & 
            (predictions['Warengruppe_Name'] == warengruppe_name)
        ]
        
        if not similar_data.empty:
            # Berechne durchschnittlichen Umsatz für ähnliche Tage
            avg_umsatz = similar_data['Umsatz'].mean()
        else:
            # Wenn keine ähnlichen Daten gefunden werden, verwende den Gesamtdurchschnitt der Warengruppe
            avg_umsatz = predictions[predictions['Warengruppe_Name'] == warengruppe_name]['Umsatz'].mean()
            # Falls immer noch keine Daten vorhanden sind, verwende den Gesamtdurchschnitt
            if pd.isna(avg_umsatz):
                avg_umsatz = predictions['Umsatz'].mean()
        
        missing_data.append({
            'id': missing_id,
            'Umsatz': avg_umsatz
        })
    
    # Erstelle DataFrame mit fehlenden Daten
    missing_df = pd.DataFrame(missing_data)
    
    # Kombiniere mit vorhandenen Daten
    submission = pd.concat([submission, missing_df], ignore_index=True)
    
    print(f"Fehlende IDs wurden mit geschätzten Umsätzen ergänzt.")

# 10. Sortiere nach der Reihenfolge der sample_submission
submission_final = sample_submission[['id']].merge(
    submission,
    on='id',
    how='left'
)

print("\nFinale Submission:")
print(submission_final.head(10))
print(f"Shape: {submission_final.shape}")

# 11. Prüfe, ob alle IDs vorhanden sind
missing_ids = set(sample_submission['id']) - set(submission_final['id'])
if missing_ids:
    print(f"\nFehlende IDs: {len(missing_ids)}")
    print(missing_ids)
else:
    print("\nAlle IDs sind vorhanden!")

# 12. Speichere als prediction.csv
submission_final.to_csv('/workspaces/bakery_sales_prediction/3_Model/prediction.csv', index=False)
print("\nSubmission gespeichert als prediction.csv")

# 13. Validierung
print("\nValidierung:")
print(f"Anzahl Zeilen sample_submission: {len(sample_submission)}")
print(f"Anzahl Zeilen prediction: {len(submission_final)}")
print(f"ID-Format korrekt: {all(len(str(id_val)) == 7 for id_val in submission_final['id'])}")
print(f"Erste 5 IDs: {submission_final['id'].head().tolist()}")
print(f"Fehlende Umsatzwerte: {submission_final['Umsatz'].isna().sum()}")

Vorhersage-Datei geladen:
        Datum Warengruppe_Name     Umsatz
0  2018-01-02             Brot  121.66526
1  2018-01-02           Kuchen  242.17667
2  2018-01-02        Croissant  118.66459
3  2018-01-02         Brötchen  275.48746
4  2018-01-02       Konditorei   87.50840
Shape: (1040, 3)

Sample submission geladen:
        id  Umsatz
0  1808011       0
1  1808021       0
2  1808031       0
3  1808041       0
4  1808051       0
Shape: (1830, 2)

ID-Typ in sample_submission: int64

Anzahl fehlender IDs vor Korrektur: 1830
Fehlende IDs wurden mit geschätzten Umsätzen ergänzt.

Finale Submission:
        id      Umsatz
0  1808011  127.117316
1  1808021  146.985748
2  1808031  136.517905
3  1808041  154.451213
4  1808051   89.801582
5  1808061  142.515203
6  1808071  129.738674
7  1808081  127.117316
8  1808091  146.985748
9  1808101  136.517905
Shape: (1830, 2)

Alle IDs sind vorhanden!

Submission gespeichert als prediction.csv

Validierung:
Anzahl Zeilen sample_submission: 1830
Anz